In [1]:
"""
notebook designed to minimize mistie for crossing radar tracks with intersecting interpretations
follows methods of Herkommer and Whitney, 1994

### psuedocode ###
1. read in pick point dataframe
2. convert points in each track to continuous linestring
3. find all intersection points for all linestrings
4. find closest 2 points in each track surrounding intersection point
5. linearly interpolate between two points to get elevation measurement for each track exactyl at intersection point
6. get actual elevation at tie point as average of two linearly interpolated measurements
7. mistie correction is achieved by a constant shift to each line based on the average mistie for all intersections on that line to the tie point averaged elevation measurement

BST 15OCT2020
python3
"""
### impots ###
import sys, os, itertools
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import interpolate
from geopandas import GeoDataFrame
from shapely.geometry import Point, LineString


%matplotlib inline
plt.rcParams['figure.figsize'] = [20, 8]

In [2]:
### functions ###
R = 6367000     # earth's radius in meters

def haversinedist_point_array(plon, plat, lonarray, latarray):
    """
    calculate the great circle distance between points on the earth (specified in decimal degrees)
    """
    plon, plat, lonarray, latarray = map(np.radians, [plon, plat, lonarray, latarray])

    dlon = lonarray - plon
    dlat = latarray - plat

    a = np.sin(dlat/2.0)**2 + np.cos(plat) * np.cos(latarray) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    m = R * c
    return m

def track_crossings(lines):
    """ 
    append track names and point to crrossings list for each track intersection
    """
    crossings = []
    for line1,line2 in  itertools.combinations(lines, 2):
        if  line1[1].intersects(line2[1]):
            crossing = line1[1].intersection(line2[1])
            if "Point" == crossing.type:
                crossings.append([line1[0],line2[0],crossing])
            elif "MultiPoint" == crossing.type:
                for pt in crossing:
                    crossings.append([line1[0],line2[0],pt])
            elif "MultiLineString" == crossing.type:
                multiLine = [line for line in crossing]
                first_coords = multiLine[0].coords[0]
                last_coords = multiLine[len(multiLine)-1].coords[1]
                crossings.append([line1[0],Point(first_coords[0], first_coords[1])])
                crossings.append([line2[0],Point(last_coords[0], last_coords[1])])
    return crossings

def interp3d(x1, y1, z1, x2, y2, z2 , x, y):
    """
    interpolate zvalue on 3d line
    """
    b1 = np.sqrt((x2 - x1)**2 + (y2 - y1 )**2)
    b2 = np.sqrt((x - x1)**2 + (y - y1)**2) 
    z =   z1 + b2*(z2 - z1) / b1
    
    return z

In [4]:
### params ###
dpath = "/mnt/c/Users/btobers/Documents/data/radar/malaspina/"
thresh = 100                # distance threshold between crossover points [m]

In [5]:
# now let's try with some malaspin data
df = pd.read_csv(dpath + "malaBed.csv")
df.head()

,file,trace,lon,lat,elev_air,elev_gnd,twtt_surf,pick_idx,twtt_bed,elev_bed,thick,elev_gnd_a,elev_bed_a
0,2015_may14_15,145,-140.196419,59.866553,421.760,-145.240,0.000004,232.0,0.000005,-217.923266,72.683266,39.450012,-33.233254
1,2015_may14_15,146,-140.197948,59.866813,423.271,-143.729,0.000004,230.0,0.000005,-213.031649,69.302649,45.812408,-23.490240
2,2015_may14_15,147,-140.200248,59.867207,425.814,-141.186,0.000004,230.0,0.000005,-210.488649,69.302649,54.026669,-15.275980
3,2015_may14_15,148,-140.201791,59.867472,426.007,-140.993,0.000004,231.0,0.000005,-211.985957,70.992957,50.180809,-20.812148
4,2015_may14_15,149,-140.204145,59.867869,423.588,-146.412,0.000004,233.0,0.000005,-219.095266,72.683266,52.160671,-20.522595


In [6]:
# lets try converting points in each track to it's own linestring then seeing where linestrings intersect

# Zip the coordinates into a point object and convert to a GeoDataFrame
geometry = [Point(xy) for xy in zip(df.lon, df.lat)]
gdf = GeoDataFrame(df, geometry=geometry)

# Aggregate these points with the GroupBy
gdf2 = gdf.groupby(["file"])["geometry"].apply(lambda x: LineString(x.tolist()))
gdf2 = GeoDataFrame(gdf2, geometry="geometry")

# compile list of lines with linestring
lines = list(gdf2["geometry"])
lines = [tuple((fname, lines[_i])) for _i, fname in enumerate(gdf2.index.values)]

In [7]:
# get all line crossings
crossings = track_crossings(lines)

In [7]:
# let's get the two surrounding points on each track to each intersection point
rem_idx = []        # indices of crossings list elements to remove if distance is above threshold
for _i, crossing in enumerate(crossings):
    pt = crossing[2]
    df_sub0 = df[df["file"]==crossing[0]]
    df_sub1 = df[df["file"]==crossing[1]]
    
    # get distance of each point in track to intersection point
    dist0 = haversinedist_point_array(df_sub0["lon"].to_numpy(), df_sub0["lat"].to_numpy(), pt.x, pt.y)
    dist1 = haversinedist_point_array(df_sub1["lon"].to_numpy(), df_sub1["lat"].to_numpy(), pt.x, pt.y)

    # make sure closest point to each intersection is below threshold distance specified
    if dist0.min() < thresh and dist1.min() < thresh:
        # sort distances to get two closes indices
        t0sort = np.argsort(dist0)
        t1sort = np.argsort(dist1)

        # get lon, lat, elev for two surrounding points on each track
        x0 = [df_sub0["lon"].iloc[t0sort[0]], df_sub0["lon"].iloc[t0sort[1]]]
        y0 = [df_sub0["lat"].iloc[t0sort[0]], df_sub0["lat"].iloc[t0sort[1]]]
        z0 = [df_sub0["elev_bed_a"].iloc[t0sort[0]], df_sub0["elev_bed_a"].iloc[t0sort[1]]]

        x1 = [df_sub1["lon"].iloc[t1sort[0]], df_sub1["lon"].iloc[t1sort[1]]]
        y1 = [df_sub1["lat"].iloc[t1sort[0]], df_sub1["lat"].iloc[t1sort[1]]]
        z1 = [df_sub1["elev_bed_a"].iloc[t1sort[0]], df_sub1["elev_bed_a"].iloc[t1sort[1]]]

        # get linearly interpolated elevation for each track at crossing point - different result if using lon/lat
        z0_cross = interp3d(x0[0],y0[0],z0[0],x0[1],y0[1],z0[1],pt.x,pt.y)
        z1_cross = interp3d(x1[0],y1[0],z1[0],x1[1],y1[1],z1[1],pt.x,pt.y)
        # z0_cross = np.interp(pt.y, y0, z0)
        # z1_cross = np.interp(pt.y, y1, z1)
        # get actual elevation at crossing point as average of two measurements
        z = (z0_cross + z1_cross) / 2

        # get difference between average value at crossing point and measured value for each track and add difference value to crossings list
        crossings[_i].append(z - z0_cross)
        crossings[_i].append(z - z1_cross)

    # if closest point further than threshold, remove crossing
    else:
        rem_idx.append(_i)

# remove necessary crossings
for idx in sorted(rem_idx, reverse = True):  
    del crossings[idx] 

In [8]:
# go through each track and get all misties
tracklist = gdf2.index.values
mistie = [0] * len(tracklist)
# iterate through crossings misties and add up total mistie for each track
for crossing in crossings:
    t0 = crossing[0]    
    t1 = crossing[1]
    idx0 = np.where(tracklist==t0)[0][0]
    idx1 = np.where(tracklist==t1)[0][0]
    # add the mistie
    mistie[idx0] += crossing[3]
    mistie[idx1] += crossing[4]
print(mistie)

[81.48221484760506, -14.102148524294421, 131.30732197322456, -242.6086203827228, 265.43887372639244, -53.4200885817882, 202.3319112193009, -258.4525816811654, 308.61900578054383, 0, 283.0675835613762, 222.77311401634267, -53.56881416426848, 19.945464330324974, 113.81661539865077, 125.91938196035514, -117.3268856522167, -83.91257793366627, 330.75304469341967, -108.56159369576599, 81.76186696171114, 290.8028915924413, 20.41422976103076, 277.3936525619307, 27.36348419908262, -1261.2567554392085, -2707.9489079414966, 40.82908112598622, 811.6428745081621, 59.44511460266551, 78.74455315479932, 1054.3368888046589, 72.96980521658936]
